In [1]:
import os
import pandas as pd
import csv
import shutil
import numpy as np
import datetime
from datetime import timedelta

# Section 1

Se crean las carpetas en blanco para guardar los nuevos archivos modificados

In [ ]:
#Crear carpetas en blanco

dir = "Datos_modificados"
if not os.path.exists(dir):
    os.makedirs(dir)

pollutants = ["CO", "NO", "NO2", "NOX", "O3", "PM2.5", "PM10", "RH", "SO2", "TMP", "WDR", "WSP"]
for pollutant in pollutants:
    pollutant_path = os.path.join(dir, pollutant)
    print(pollutant_path)
    os.makedirs(pollutant_path)
    year = 2005
    for y in range(19):
        year_path = os.path.join(pollutant_path, str(year+y))
        os.makedirs(year_path)
        print(year, year_path, y)



# Section 2

Los archivos descargados de la página aire y salud tiene una fila al inicio y dos al final que no pertenecen a la tabla de valores (el título e información pareciera de html),
por lo que se eliminan para que se puedan leer como dataframes con pandas, de otra forma causa error

In [3]:
def delete_rows(dir, new_dir, file_path):
    filas = []
    dataframe = pd.read_csv(file_path)
    num_rows = len(dataframe)
    #print(num_rows, file_path)
    with open(file_path, 'r', newline='', encoding='utf-8') as archivo_csv:
        lector_csv = csv.reader(archivo_csv)
        for row in lector_csv:
            if "Promedio" in row[0]:
                print(row[0])    
            elif "<br" in row[0]:
                print(row[0])  
            elif "<b" in row[0]:
                print(row[0])          
            else:
                filas.append(row)


    subpath = file_path[len(dir):len(file_path)]
    new_path = os.path.join(new_dir, subpath)
    
    with open(new_path, 'w', newline='', encoding='utf-8') as archivo_csv_nuevo:
        escritor_csv = csv.writer(archivo_csv_nuevo)
        escritor_csv.writerows(filas)

    dataframe = pd.read_csv(new_path)
    num_rows = len(dataframe)
    #print(num_rows)
    return dataframe, new_path


Se cambia el nombre de todos los archivos descargados (todos dicen Promedio_contaminanteX), por el nombre del contaminante_el mes_año

In [4]:
dir = "Datos_modificados/"
#dir = "Datos Aire y Salud/"
new_dir = "Datos_modificados/"


for actual_folder, folders, files in os.walk(dir):
    for file_name in files:
        file_path = os.path.join(actual_folder, file_name)
        if file_path.endswith(".csv"):
            sub_folders = file_path.split("/")
            sub_folders = sub_folders[len(sub_folders)-1].split("\\")
            variable = sub_folders[len(sub_folders)-3]
            variable = variable.replace(".","")
            year = sub_folders[len(sub_folders)-2]
            print(file_path, variable, year)

            try:
                dataframe, new_path = delete_rows(dir, new_dir, file_path)
            except:                
                subpath = file_path[len(dir):len(file_path)]
                path = os.path.join(new_dir, subpath)
                new_path = os.path.split(path)
                new_path = new_path[0]
                new_path = shutil.copy(file_path, new_path)
                dataframe = pd.read_csv(new_path)
            #if file_name.startswith("Promedio"):
            if True:
                months = {"01":"JAN", "02":"FEB", "03":"MAR", "04":"APR", "05":"MAY", "06":"JUN", "07":"JUL", "08":"AUG", "09":"SEP", "10":"OCT", "11":"NOV", "12":"DEC"}
                dataframe['Fecha'] = dataframe['Fecha'].str.replace('-', '/')
                date = dataframe.iloc[0].Fecha
                num_month = date.split("/")
                month = months[num_month[1]]

                new_name = os.path.join(new_dir,variable)
                new_name = os.path.join(new_name, year)
                name = variable + "_" +month+"_"+year + ".csv"
                new_name = os.path.join(new_name, name)
                os.rename(new_path,new_name)


Datos_modificados/CO\2024\CO_MAY_2024.csv CO 2024
Datos_modificados/NO\2024\NO_MAY_2024.csv NO 2024
Datos_modificados/NO2\2024\NO2_MAY_2024.csv NO2 2024
Datos_modificados/NOX\2024\NOX_MAY_2024.csv NOX 2024
Datos_modificados/O3\2024\O3_MAY_2024.csv O3 2024
Datos_modificados/PM10\2024\PM10_MAY_2024.csv PM10 2024
Datos_modificados/PM25\2024\PM25_MAY_2024.csv PM25 2024
Datos_modificados/RH\2024\RH_MAY_2024.csv RH 2024
Datos_modificados/SO2\2024\SO2_MAY_2024.csv SO2 2024
Datos_modificados/TMP\2024\TMP_MAY_2024.csv TMP 2024
Datos_modificados/WDR\2024\WDR_MAY_2024.csv WDR 2024
Datos_modificados/WSP\2024\WSP_MAY_2024.csv WSP 2024


Se juntan POR CONTAMINANTE en un solo dataframe los datos de todos los meses y todos los año

In [6]:
root_dir = "Datos_modificados" #new_dir
files_path = "Archivos_contaminantes_juntas"

if not os.path.exists(files_path):
    os.makedirs(files_path)
    
dataframes_cont = {"CO":[], "SO2":[], "NO":[], "NO2":[], "NOX":[], "PM10":[], "PM25":[],
				 "O3":[], "RH":[], "WSP":[], "WDR":[], "TMP":[]}
# Lista de contaminantes (nombres de carpetas en la raíz)
contaminantes = os.listdir(root_dir)

# Años y meses esperados
anios = list(range(2005, 2025))
meses = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
                
# Guardamos archivos problemáticos
errores = []

for cont in contaminantes:
	cont = cont.replace(".","")
	cont_path = os.path.join(root_dir, cont)

	# Nos aseguramos de que sea una carpeta
	if not os.path.isdir(cont_path):
		print("ERRRRRROOOOOR", cont_path)
		continue
    
	for anio in anios:
		anio_path = os.path.join(cont_path, str(anio))
		
		for mes in meses:
			archivo = f"{cont}_{mes}_{anio}.csv"
			archivo_path = os.path.join(anio_path, archivo)
			
			if not os.path.exists(archivo_path):
				errores.append((cont, anio, mes, "NO ENCONTRADO"))
				print("No ENCONTRADO")
				continue
			
			try:
				df = pd.read_csv(archivo_path)
				dataframes_cont[cont].append(df)

			except Exception as e:
				errores.append((cont, anio, mes, f"ERROR LECTURA: {e}"))

# Mostrar resultados
if errores:
    print("Archivos faltantes o con error:")
    for e in errores:
        print(f"Contaminante: {e[0]}, Año: {e[1]}, Mes: {e[2]}, Error: {e[3]}")
else:
    print("¡Todos los archivos fueron encontrados y leídos correctamente!")

for cont in contaminantes:
	cont = cont.replace(".","")
	df_final = pd.concat(dataframes_cont[cont], ignore_index=True, sort=False)
	cont_name = cont + ".csv"
	file_name = os.path.join(files_path,cont_name)
	df_final.to_csv(file_name, index=False)


No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENCONTRADO
No ENC

Se cambian los datos vacios con valor de "nr" por NaN, y se separa la fecha en columnas de día, mes, año, y se crea una nueva con los datos de la fecha y la hora para poder ordenarlos de forma ascendente

# Section 3

Se guarda toda la info de todos los contaminantes en un solo dataframe

In [7]:
dir = "Archivos_contaminantes_juntas"
dir2 = "Archivos_contaminantes_juntas2"
#final_filename = "data_31May_2024_all_stations.csv"
#final_filename = "data_oct_dic_2024_all_stations.csv"
final_filename = "data_2005_dic2024_all_stations.csv"
if not os.path.exists(dir2):
    os.makedirs(dir2)

files = os.listdir(dir)
dataframes = []
for file_name in files:
	file_dir = os.path.join(dir, file_name)
	df = pd.read_csv(file_dir, low_memory=False)
	df.replace('nr', np.nan, inplace=True)
	columns = df.columns
	for column in columns:
		if 'Unnamed' in column:
			df = df.drop(column, axis=1)
	df['Fecha'] = df['Fecha'].str.replace('-', '/')
	df[['dia', 'mes', 'year']] = df['Fecha'].str.split('/', expand=True)     #separamos la fecha para poder unirla con la hora
	df['Hora'] = pd.to_datetime(df['Hora'], unit='h')
	df['Hora'] = df['Hora'] - timedelta(hours=1)
	# Formatear la columna 'horas' en el formato deseado
	df['Hora'] = df['Hora'].dt.strftime('%H:%M')
	df['fecha_hora'] = df["year"].astype(str) +"/"+ df["mes"].astype(str)+"/"+df["dia"].astype(str) + ' ' + df['Hora'].astype(str)
	pollutant = file_name.split(".")[0]
	df["pollutant"] = pollutant
	columna_especifica="fecha_hora"
	# Verificar si la columna tiene valores NaN
	tiene_nan = df[columna_especifica].isnull().any()
	# Mostrar el resultado
	if tiene_nan:
		print(f"{pollutant}. La columna '{columna_especifica}' tiene valores NaN.")
	else:
		print(f"{pollutant}. La columna '{columna_especifica}' no tiene valores NaN.")

	df.reset_index(drop=True, inplace=True)
	file_dir2 = os.path.join(dir2, file_name)
	df.to_csv(file_dir2, index=False) 
	dataframes.append(df)
df_final = pd.concat(dataframes, ignore_index=False, sort=False)
df_final.to_csv(final_filename, index=False)
df = pd.read_csv(final_filename)
del df["LVI"]
del df["SS1"]
if "Unnamed: 0" in df.keys():
    del df["Unnamed: 0"]
df.to_csv(final_filename , index=False)
df

C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\2122686118.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('nr', np.nan, inplace=True)
C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\2122686118.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('nr', np.nan, inplace=True)
C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\2122686118.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.

CO. La columna 'fecha_hora' no tiene valores NaN.
NO. La columna 'fecha_hora' no tiene valores NaN.
NO2. La columna 'fecha_hora' no tiene valores NaN.
NOX. La columna 'fecha_hora' no tiene valores NaN.
O3. La columna 'fecha_hora' no tiene valores NaN.
PM10. La columna 'fecha_hora' no tiene valores NaN.
PM25. La columna 'fecha_hora' no tiene valores NaN.


C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\2122686118.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('nr', np.nan, inplace=True)
C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\2122686118.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('nr', np.nan, inplace=True)
C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\2122686118.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.

RH. La columna 'fecha_hora' no tiene valores NaN.
SO2. La columna 'fecha_hora' no tiene valores NaN.
TMP. La columna 'fecha_hora' no tiene valores NaN.
WDR. La columna 'fecha_hora' no tiene valores NaN.
WSP. La columna 'fecha_hora' no tiene valores NaN.


,Fecha,Hora,ACO,AJM,AJU,ARA,ATI,AZC,BJU,CAM,...,UAX,UIZ,VAL,VIF,XAL,dia,mes,year,fecha_hora,pollutant
0,01/05/2024,00:00,NaN,NaN,NaN,NaN,0.71,NaN,0.97,0.55,...,0.78,0.89,NaN,0.73,NaN,1,5,2024,2024/05/01 00:00,CO
1,01/05/2024,01:00,NaN,NaN,NaN,NaN,0.70,NaN,0.66,0.61,...,0.71,0.91,NaN,0.46,NaN,1,5,2024,2024/05/01 01:00,CO
2,01/05/2024,02:00,NaN,NaN,NaN,NaN,0.57,NaN,0.60,0.97,...,0.68,0.69,NaN,0.70,NaN,1,5,2024,2024/05/01 02:00,CO
3,01/05/2024,03:00,NaN,NaN,NaN,NaN,0.61,NaN,NaN,0.96,...,0.60,0.65,NaN,0.75,NaN,1,5,2024,2024/05/01 03:00,CO
4,01/05/2024,04:00,NaN,NaN,NaN,NaN,0.63,NaN,0.62,0.80,...,0.61,0.59,NaN,0.73,NaN,1,5,2024,2024/05/01 04:00,CO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,31/05/2024,19:00,NaN,2.5,1.9,NaN,NaN,NaN,1.30,NaN,...,NaN,NaN,NaN,2.10,NaN,31,5,2024,2024/05/31 19:00,WSP
8924,31/05/2024,20:00,NaN,1.7,1.6,NaN,NaN,NaN,1.70,NaN,...,NaN,NaN,NaN,1.70,NaN,31,5,2024,2024/05/31 20:00,WSP
8925,31/05/2024,21:00,NaN,2.5,2.2,NaN,NaN,NaN,1.30,NaN,...,NaN,NaN,NaN,1.30,NaN,31,5,2024,2024/05/31 21:00,WSP
8926,31/05/2024,22:00,NaN,2.1,2.5,NaN,NaN,NaN,0.40,NaN,...,NaN,NaN,NaN,1.10,NaN,31,5,2024,2024/05/31 22:00,WSP


In [28]:
df

,Fecha,Hora,ACO,AJM,AJU,ARA,ATI,AZC,BJU,CAM,...,UAX,UIZ,VAL,VIF,XAL,dia,mes,year,fecha_hora,pollutant
353,15/05/2024,17:00,NaN,0.74,NaN,NaN,0.84,NaN,0.68,NaN,...,0.72,NaN,NaN,0.55,NaN,15,5,2024,2024/05/15 17:00,CO
354,15/05/2024,18:00,NaN,NaN,NaN,NaN,1.20,NaN,0.70,NaN,...,0.70,NaN,NaN,0.60,NaN,15,5,2024,2024/05/15 18:00,CO
355,15/05/2024,19:00,NaN,0.69,NaN,NaN,1.11,NaN,0.73,NaN,...,0.74,NaN,NaN,0.49,NaN,15,5,2024,2024/05/15 19:00,CO
356,15/05/2024,20:00,NaN,0.72,NaN,NaN,1.02,NaN,0.70,NaN,...,0.77,NaN,NaN,0.46,NaN,15,5,2024,2024/05/15 20:00,CO
357,15/05/2024,21:00,NaN,0.85,NaN,NaN,1.10,NaN,0.76,NaN,...,0.78,NaN,NaN,0.53,NaN,15,5,2024,2024/05/15 21:00,CO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,31/05/2024,19:00,NaN,2.50,1.9,NaN,NaN,NaN,1.30,NaN,...,NaN,NaN,NaN,2.10,NaN,31,5,2024,2024/05/31 19:00,WSP
8924,31/05/2024,20:00,NaN,1.70,1.6,NaN,NaN,NaN,1.70,NaN,...,NaN,NaN,NaN,1.70,NaN,31,5,2024,2024/05/31 20:00,WSP
8925,31/05/2024,21:00,NaN,2.50,2.2,NaN,NaN,NaN,1.30,NaN,...,NaN,NaN,NaN,1.30,NaN,31,5,2024,2024/05/31 21:00,WSP
8926,31/05/2024,22:00,NaN,2.10,2.5,NaN,NaN,NaN,0.40,NaN,...,NaN,NaN,NaN,1.10,NaN,31,5,2024,2024/05/31 22:00,WSP


In [20]:
df = df[df.fecha_hora>="2024/05/15 17:00"]

Número de estaciones de las que se tiene información

In [8]:
len(df.keys())-7

48

# Section 4

## No ejecutar si se hace desde cero (es decir, no hay información antigua con la que juntar la nueva)
Unir información nueva con antigua

In [ ]:
old_df = pd.read_csv("data_sep_2024_all_stations.csv")
old_df["fecha_hora"]

In [ ]:
dir = "data_oct_dic_2024_all_stations.csv"
final_filename = "data_feb_dic_2024_all_stations.csv"
df = pd.read_csv(dir)
df["fecha_hora"]

0        2024/10/01 00:00
1        2024/10/01 01:00
2        2024/10/01 02:00
3        2024/10/01 03:00
4        2024/10/01 04:00
               ...       
26491    2024/12/31 19:00
26492    2024/12/31 20:00
26493    2024/12/31 21:00
26494    2024/12/31 22:00
26495    2024/12/31 23:00
Name: fecha_hora, Length: 26496, dtype: object

In [11]:
# Ejecutar solo si se tiene información de la parte restante de un mes que ya estaba
# Hay que borrar los datos que ya se tienen de ese mes

#dir = "data_31May_2024_all_stations.csv"

#dir = final_filename
df = pd.read_csv(dir, dtype={'fecha_hora': 'str'})
# Convierte la columna 'fecha' a tipo timestamp
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'], format='%d/%m/%Y')
#df['fecha_hora'] = pd.to_datetime(df['fecha_hora'], format='%Y/%m/%d %H:%M:%S')
fecha_limite = pd.Timestamp('2024/05/15 17:00')
new_set = df[df['fecha_hora'] > fecha_limite].copy()
new_set['fecha_hora'] = new_set['fecha_hora'].dt.strftime('%Y/%m/%d %H:%M')
new_set["fecha_hora"]
new_set.to_csv(final_filename, index=False)

In [12]:
len(old_df)+len(new_set)

2081685

In [13]:
df_concat = pd.concat([old_df, new_set])
print(len(df_concat))
df_concat.to_csv(final_filename)

2081685


# Section 5

In [ ]:
df = pd.read_csv(final_filename)
estaciones = df.keys()
borrar = ['Fecha', 'Hora', 'dia', 'mes', 'year', 'fecha_hora', 'pollutant']
estaciones = [elemento for elemento in estaciones if elemento not in borrar]
len(estaciones)

48

Revisar los nombres de las carpetas donde se guardarán y de donde se tomarán los datos

In [15]:
dir_to_save = "datos_por_estacion_marzo2024"
if not os.path.exists(dir_to_save):
    os.makedirs(dir_to_save)
    

pollutants = ["CO", "NO", "NOX", "NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP"]

for estacion in estaciones:
  columnas_interes = ["fecha_hora", "pollutant"]
  columnas_interes.append(estacion)
  datos_modelos = df[columnas_interes]
  datos_modelos = datos_modelos.rename(columns={estacion: "value"})
  dataframes = []
  for pollutant in pollutants:
    data = datos_modelos[datos_modelos.pollutant==pollutant]
    data = data.set_index("fecha_hora")
    data.index.name = 'date'
    data.drop('pollutant', axis=1, inplace=True)
    data = data.rename(columns={"value":pollutant})
    dataframes.append(data)

  df_merged = pd.DataFrame()
  columnas = ['date']
  df_merged = pd.DataFrame(columns=columnas)
  for dataframe in dataframes:
    df_merged = pd.merge(df_merged, dataframe, on='date', how='outer')

  df_merged = df_merged.reset_index()
  df_merged = df_merged.drop(columns=["index"])
  df_merged_sin_fecha = df_merged.drop(columns=["date"])
  indices_filas_vacias = df_merged_sin_fecha[df_merged_sin_fecha.isna().all(axis=1)].index
  df_filtrado = df_merged.drop(indices_filas_vacias)
  df_filtrado = df_filtrado.set_index("date")
  df_filtrado= df_filtrado.sort_values(by='date') 
  df_filtrado = df_filtrado.reset_index()
  try:
    df_filtrado['date'] = df_filtrado['date'].dt.strftime('%Y/%m/%d %H:%M')
  #"""
  except:
    for i in range(len(df_filtrado)):
      try: 
        fecha = df_filtrado.iloc[i]['date']
        if type(fecha) == str: 
                if len(fecha) == 19:
                    # Cambiar el formato de la fecha a 'YYYY/MM/DD HH:MM'
                    fecha_dt = datetime.datetime.strptime(fecha, '%Y/%m/%d %H:%M:%S')
                    # Formatear la fecha al formato deseado (sin segundos)
                    df_filtrado.iloc[i]['date'] = fecha_dt.strftime('%Y/%m/%d %H:%M')
                elif len(fecha) == 16:
                    #Ya esta con el formato 'YYYY/MM/DD HH:MM'
                    df_filtrado.iloc[i]['date'] = pd.to_datetime(fecha, format='%Y/%m/%d %H:%M')
      except Exception as e:
        print(e)
        print(i,  type(df_filtrado.iloc[i]['date']))
        print(df_filtrado.iloc[i])
  #"""
  name = estacion + ".csv"
  dir_file = os.path.join(dir_to_save,name)
  df_filtrado.to_csv(dir_file, index=False)

C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\845064567.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.iloc[i]['date'] = pd.to_datetime(fecha, format='%Y/%m/%d %H:%M')
C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\845064567.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.iloc[i]['date'] = pd.to_datetime(fecha, format='%Y/%m/%d %H:%M')
C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\845064567.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

Se borran las carpetan que ya no nos sirven

In [16]:
dir = "Archivos_contaminantes_juntas"
#shutil.rmtree(dir)
dir = "Archivos_contaminantes_juntas2"
#shutil.rmtree(dir)
dir = "datos_Modificados"
shutil.rmtree(dir)

#os.remove(final_filename)

# Section 6

In [2]:
import pandas as pd
import os

In [3]:
#dir1 = r"C:\Users\valer\Documents\CIC\doctorado\air_pollution\preprocessing_airpollution_data\data_31May_2024_all_stations.csv"
dir1 = r"C:\Users\valer\Documents\CIC\doctorado\air_pollution\preprocessing_airpollution_data\data_sep_2024_all_stations.csv.csv"
dir2 = r"C:\Users\valer\Documents\CIC\doctorado\air_pollution\preprocessing_airpollution_data\data_jun_sep_2024_all_stations.csv"
d1 = pd.read_csv(dir1)
d2 = pd.read_csv(dir2)

In [5]:
union = pd.concat([d1, d2], ignore_index=True)

# Guardar el resultado en un nuevo archivo CSV
union.to_csv('data_sep_2024_all_stations.csv', index=False)


In [38]:
df

,Fecha,Hora,ACO,AJM,AJU,ARA,ATI,AZC,BJU,CAM,...,UAX,UIZ,VAL,VIF,XAL,dia,mes,year,fecha_hora,pollutant
353,15/05/2024,17:00,NaN,0.74,NaN,NaN,0.84,NaN,0.68,NaN,...,0.72,NaN,NaN,0.55,NaN,15,5,2024,2024/05/15 17:00,CO
354,15/05/2024,18:00,NaN,NaN,NaN,NaN,1.20,NaN,0.70,NaN,...,0.70,NaN,NaN,0.60,NaN,15,5,2024,2024/05/15 18:00,CO
355,15/05/2024,19:00,NaN,0.69,NaN,NaN,1.11,NaN,0.73,NaN,...,0.74,NaN,NaN,0.49,NaN,15,5,2024,2024/05/15 19:00,CO
356,15/05/2024,20:00,NaN,0.72,NaN,NaN,1.02,NaN,0.70,NaN,...,0.77,NaN,NaN,0.46,NaN,15,5,2024,2024/05/15 20:00,CO
357,15/05/2024,21:00,NaN,0.85,NaN,NaN,1.10,NaN,0.76,NaN,...,0.78,NaN,NaN,0.53,NaN,15,5,2024,2024/05/15 21:00,CO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,31/05/2024,19:00,NaN,2.50,1.9,NaN,NaN,NaN,1.30,NaN,...,NaN,NaN,NaN,2.10,NaN,31,5,2024,2024/05/31 19:00,WSP
8924,31/05/2024,20:00,NaN,1.70,1.6,NaN,NaN,NaN,1.70,NaN,...,NaN,NaN,NaN,1.70,NaN,31,5,2024,2024/05/31 20:00,WSP
8925,31/05/2024,21:00,NaN,2.50,2.2,NaN,NaN,NaN,1.30,NaN,...,NaN,NaN,NaN,1.30,NaN,31,5,2024,2024/05/31 21:00,WSP
8926,31/05/2024,22:00,NaN,2.10,2.5,NaN,NaN,NaN,0.40,NaN,...,NaN,NaN,NaN,1.10,NaN,31,5,2024,2024/05/31 22:00,WSP


In [134]:

dir = r"C:\Users\Valeria\Documents\CIC\air_pollution_data\datos_por_estacion_2005_dic2024"
estacion = "MER"
file = f"{estacion}.csv"
file_path = os.path.join(dir, file)
df_ = pd.read_csv(r"C:\Users\Valeria\Documents\CIC\air_pollution_data\datos_por_estacion_2005_dic2024\MER.csv")
df_
#df_['date']  = pd.to_datetime(df_['date'] , format='%Y/%m/%d %H:%M')
#type(df_.date.iloc[0])

,date,CO,NO,NOX,NO2,O3,PM10,PM25,RH,SO2,TMP,WDR,WSP
0,2005-01-01 00:00:00,6.4,182.0,277.0,95.0,0.0,136.0,99.0,55.0,20.0,11.0,134.0,0.2
1,2005-01-01 01:00:00,4.2,107.0,193.0,85.0,0.0,158.0,114.0,55.0,20.0,10.0,NaN,NaN
2,2005-01-01 02:00:00,3.7,79.0,156.0,77.0,0.0,169.0,140.0,57.0,18.0,10.0,NaN,NaN
3,2005-01-01 03:00:00,3.5,80.0,148.0,68.0,0.0,161.0,137.0,58.0,17.0,9.0,NaN,NaN
4,2005-01-01 04:00:00,3.6,77.0,143.0,67.0,1.0,182.0,142.0,60.0,17.0,8.0,56.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163294,2024-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,19.0,17.0,3.8
163295,2024-12-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,17.0,6.0,2.5
163296,2024-12-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,16.0,321.0,3.0
163297,2024-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,16.0,299.0,2.7


In [119]:

from datetime import datetime
dir = r"C:\Users\Valeria\Documents\CIC\air_pollution_data\datos_por_estacion_2005_dic2024"
dir2 = r"C:\Users\Valeria\Documents\CIC\air_pollution\preprocessing_airpollution_data\datos_por_estacion_marzo2024"
for estacion in estaciones:
	file = f"{estacion}.csv"
	print(estacion)
	file_path = os.path.join(dir, file)
	df_ = pd.read_csv(file_path)
	df_['date']  = pd.to_datetime(df_['date'] , format='%Y/%m/%d %H:%M')
	df0 = df_[df_.date<datetime(2024, 5, 15, 17, 0)]#"2024/05/15 17:00"]
	df2 = df_[df_.date>=datetime(2024, 6, 1, 0, 0)]#"2024/05/15 17:00"]
	file_path2 = os.path.join(dir2, file)
	df1 = pd.read_csv(file_path2)
	df1['date']  = pd.to_datetime(df1['date'] , format='%Y/%m/%d %H:%M')
	df1 = df1[df1.date>=datetime(2024, 5, 15, 17, 0)] #"2024/05/15 17:00"]
	df3 = pd.concat([df0, df1, df2], ignore_index=True, sort=False)
	df3.to_csv(file_path, index=False)

ACO


C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\507842964.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df3 = pd.concat([df0, df1, df2], ignore_index=True, sort=False)


AJM
AJU
ARA
ATI


C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\507842964.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df3 = pd.concat([df0, df1, df2], ignore_index=True, sort=False)


AZC


...

TLI
TPN


C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\507842964.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df3 = pd.concat([df0, df1, df2], ignore_index=True, sort=False)


UAX
UIZ
VAL


C:\Users\Valeria\AppData\Local\Temp\ipykernel_24104\507842964.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df3 = pd.concat([df0, df1, df2], ignore_index=True, sort=False)


VIF
XAL


In [130]:
df0

,date,CO,NO,NOX,NO2,O3,PM10,PM25,RH,SO2,TMP,WDR,WSP
0,2005-01-01 00:00:00,4.3,102.0,175.0,72.0,6.0,355.0,NaN,51.0,34.0,11.0,349.0,0.3
1,2005-01-01 01:00:00,5.1,161.0,233.0,73.0,8.0,502.0,NaN,53.0,43.0,10.0,292.0,0.2
2,2005-01-01 02:00:00,6.0,206.0,277.0,71.0,6.0,441.0,NaN,54.0,46.0,10.0,279.0,0.1
3,2005-01-01 03:00:00,4.8,133.0,204.0,70.0,7.0,409.0,NaN,57.0,45.0,9.0,41.0,0.3
4,2005-01-01 04:00:00,5.7,168.0,239.0,71.0,9.0,488.0,NaN,59.0,45.0,8.0,343.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151480,2024-05-15 12:00:00,NaN,2.0,20.0,18.0,78.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151481,2024-05-15 13:00:00,NaN,2.0,23.0,21.0,86.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151482,2024-05-15 14:00:00,NaN,1.0,21.0,19.0,95.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151483,2024-05-15 15:00:00,NaN,2.0,32.0,30.0,72.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [105]:
df_[(df_.date>="2024/05/29 17:00") & (df_.date<="2024/05/31 23:00")]

,date,CO,NO,NOX,NO2,O3,PM10,PM25,RH,SO2,TMP,WDR,WSP
151807,2024-05-29 17:00:00,0.36,4.0,27.0,23.0,44.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151808,2024-05-29 18:00:00,0.37,5.0,29.0,24.0,41.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151809,2024-05-29 19:00:00,0.53,11.0,46.0,34.0,29.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151810,2024-05-29 20:00:00,0.60,5.0,47.0,42.0,19.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151811,2024-05-29 21:00:00,0.74,12.0,59.0,48.0,12.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151812,2024-05-29 22:00:00,0.85,6.0,56.0,50.0,8.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151813,2024-05-29 23:00:00,1.01,21.0,78.0,57.0,3.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
151814,2024-05-30 00:00:00,0.99,28.0,82.0,54.0,3.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151815,2024-05-30 01:00:00,0.75,12.0,59.0,47.0,4.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151816,2024-05-30 02:00:00,0.69,12.0,57.0,45.0,5.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [124]:
df1[df1.date>="2024/05/29 17:00"]

,date,CO,NO,NOX,NO2,O3,PM10,PM25,RH,SO2,TMP,WDR,WSP
339,2024-05-29 17:00:00,0.36,4.0,27.0,23.0,44.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
340,2024-05-29 18:00:00,0.37,5.0,29.0,24.0,41.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
341,2024-05-29 19:00:00,0.53,11.0,46.0,34.0,29.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
342,2024-05-29 20:00:00,0.60,5.0,47.0,42.0,19.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
343,2024-05-29 21:00:00,0.74,12.0,59.0,48.0,12.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
344,2024-05-29 22:00:00,0.85,6.0,56.0,50.0,8.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
345,2024-05-29 23:00:00,1.01,21.0,78.0,57.0,3.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
346,2024-05-30 00:00:00,0.99,28.0,82.0,54.0,3.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
347,2024-05-30 01:00:00,0.75,12.0,59.0,47.0,4.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
348,2024-05-30 02:00:00,0.69,12.0,57.0,45.0,5.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [103]:
print(len(df_[(df_.date>="2024/05/29 17:00") & (df_.date<="2024/05/31 23:00")]))
print(len(df1[df1.date>="2024/05/29 17:00"]))
print(len(df_[(df_.date>="2024/05/29 17:00") & (df_.date<="2024/05/31 23:00")]))

55
55
55


In [107]:
df3[(df3.date>=datetime(2024, 5, 29, 17, 0)) & (df3.date<=datetime(2024, 5, 31, 23, 0))]

,date,CO,NO,NOX,NO2,O3,PM10,PM25,RH,SO2,TMP,WDR,WSP
151807,2024-05-29 17:00:00,0.36,4.0,27.0,23.0,44.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151808,2024-05-29 18:00:00,0.37,5.0,29.0,24.0,41.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151809,2024-05-29 19:00:00,0.53,11.0,46.0,34.0,29.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151810,2024-05-29 20:00:00,0.60,5.0,47.0,42.0,19.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151811,2024-05-29 21:00:00,0.74,12.0,59.0,48.0,12.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152234,2024-05-31 19:00:00,0.37,10.0,36.0,27.0,19.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
152235,2024-05-31 20:00:00,0.90,11.0,62.0,51.0,22.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
152236,2024-05-31 21:00:00,1.02,41.0,88.0,48.0,4.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
152237,2024-05-31 22:00:00,0.75,25.0,64.0,39.0,7.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [120]:
df3[df3.date==datetime(2024, 5, 29, 17, 0)]

,date,CO,NO,NOX,NO2,O3,PM10,PM25,RH,SO2,TMP,WDR,WSP
151807,2024-05-29 17:00:00,0.36,4.0,27.0,23.0,44.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [121]:
df3[(df3.date>="2024/05/29 17:00") & (df3.date<="2024/05/31 23:00")]

,date,CO,NO,NOX,NO2,O3,PM10,PM25,RH,SO2,TMP,WDR,WSP
151807,2024-05-29 17:00:00,0.36,4.0,27.0,23.0,44.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151808,2024-05-29 18:00:00,0.37,5.0,29.0,24.0,41.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151809,2024-05-29 19:00:00,0.53,11.0,46.0,34.0,29.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151810,2024-05-29 20:00:00,0.60,5.0,47.0,42.0,19.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151811,2024-05-29 21:00:00,0.74,12.0,59.0,48.0,12.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151812,2024-05-29 22:00:00,0.85,6.0,56.0,50.0,8.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151813,2024-05-29 23:00:00,1.01,21.0,78.0,57.0,3.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
151814,2024-05-30 00:00:00,0.99,28.0,82.0,54.0,3.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151815,2024-05-30 01:00:00,0.75,12.0,59.0,47.0,4.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
151816,2024-05-30 02:00:00,0.69,12.0,57.0,45.0,5.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [122]:
df3[df3.date=="2024/05/29 17:00"]

,date,CO,NO,NOX,NO2,O3,PM10,PM25,RH,SO2,TMP,WDR,WSP
151807,2024-05-29 17:00:00,0.36,4.0,27.0,23.0,44.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
